# kdotp-symmetry

## Setup

In [2]:
%%capture
%%bash
pip install kdotp-symmetry

In [6]:
import kdotp_symmetry as kp
import sympy as sp
import symmetry_representation as sr

   ## Creating symmetry operations

We will create a basis for the $\mathbf{k}\cdot\mathbf{p}$ model of TaAs$_2$. First, we define (by hand) the appropriate symmetry operations and representations.

* Rotation around $y$ 
    * rotation matrix (reduced coordinates):
    $$ C_{2y} = \begin{pmatrix} 0 & 1 & 0\\1 & 0 & 0 \\0 & 0 & 1 \end{pmatrix}$$
    * representation matrix:
    $$ C_{2y} =  \operatorname{diag}(i, -i, i, -i)$$
* Inversion
    * rotation matrix (reduced coordinates):
    $$ P = - \mathbb{1}_{3\times3} $$
    * representation matrix:
    $$ P = \operatorname{diag}(1, 1, -1, -1) $$
* Time-reversal
    * rotation matrix:
    $$ \mathcal{T} = \mathbb{1} $$
    * representation matrix:
    $$ \mathcal{T} = \begin{pmatrix} 0 & -1 & 0 & 0\\1&0&0&0\\0&0&0&-1\\0&0&1&0 \end{pmatrix} \hat{K} $$

In [10]:
c_2y = sr.SymmetryOperation(
    rotation_matrix=sp.Matrix([[0, 1, 0], [1, 0, 0], [0, 0, 1]]),
    repr_matrix=sp.diag(sp.I, -sp.I, sp.I, -sp.I))

In [11]:
inversion = sr.SymmetryOperation(
    rotation_matrix=-sp.eye(3), repr_matrix=sp.diag(1, 1, -1, -1))

In [12]:
time_reversal = sr.SymmetryOperation(
    rotation_matrix=sp.eye(3),
    repr_matrix=sp.Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 0, -1],
                           [0, 0, 1, 0]]),
    repr_has_cc=True)

Notice that we are now using ``sympy`` matrices throughout. That is, all the inputs given here need to be analytically exact.

## Obtaining the k.p basis

First, we need to choose a basis for the $k$-space function and Hermitian matrices:

In [14]:
kp.monomial_basis?

In [16]:
kp.monomial_basis(1)

[kx, ky, kz]

In [18]:
kp.hermitian_basis(2)

[Matrix([
 [1, 0],
 [0, 0]]), Matrix([
 [0, 0],
 [0, 1]]), Matrix([
 [0, 1],
 [1, 0]]), Matrix([
 [0, -I],
 [I,  0]])]

The ``symmetric_hamiltonian`` function does the heavy lifting:

In [19]:
kp.symmetric_hamiltonian?

In [29]:
for i in range(3):
    print('\nOrder:', i)
    print(
        kp.symmetric_hamiltonian(c_2y, inversion, time_reversal,
                                 expr_basis=kp.monomial_basis(i),
                                 repr_basis=kp.hermitian_basis(4)))


Order: 0
[Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]]), Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])]

Order: 1
[Matrix([
[      0,        0, kx + ky,        0],
[      0,        0,       0, -kx - ky],
[kx + ky,        0,       0,        0],
[      0, -kx - ky,       0,        0]]), Matrix([
[          0,           0, -I*kx - I*ky,            0],
[          0,           0,            0, -I*kx - I*ky],
[I*kx + I*ky,           0,            0,            0],
[          0, I*kx + I*ky,            0,            0]]), Matrix([
[      0,       0,       0, kx - ky],
[      0,       0, kx - ky,       0],
[      0, kx - ky,       0,       0],
[kx - ky,       0,       0,       0]]), Matrix([
[          0,            0,           0, -I*kx + I*ky],
[          0,            0, I*kx - I*ky,            0],
[          0, -I*kx + I*ky,           0,            0],
[I*kx - I*ky,            0,           0,            0]]), Matrix([
[ 0,   0, kz,   0],
[ 0,   0,  

Drawback: The working basis size grows very quickly when the input basis becomes larger. For example, the following is much slower:

In [30]:
kp.symmetric_hamiltonian(
    c_2y,
    inversion,
    time_reversal,
    expr_basis=kp.monomial_basis(*range(3)),
    repr_basis=kp.hermitian_basis(4))

[Matrix([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0]]), Matrix([
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1]]), Matrix([
 [      0,        0, kx + ky,        0],
 [      0,        0,       0, -kx - ky],
 [kx + ky,        0,       0,        0],
 [      0, -kx - ky,       0,        0]]), Matrix([
 [          0,           0, -I*kx - I*ky,            0],
 [          0,           0,            0, -I*kx - I*ky],
 [I*kx + I*ky,           0,            0,            0],
 [          0, I*kx + I*ky,            0,            0]]), Matrix([
 [      0,       0,       0, kx - ky],
 [      0,       0, kx - ky,       0],
 [      0, kx - ky,       0,       0],
 [kx - ky,       0,       0,       0]]), Matrix([
 [          0,            0,           0, -I*kx + I*ky],
 [          0,            0, I*kx - I*ky,            0],
 [          0, -I*kx + I*ky,           0,            0],
 [I*kx - I*ky,            0,           0,            0]]), Matrix([
 [ 0,   0, kz,   0],
 [ 0,  

As a result, calculating the k.p model in this way becomes unfeasible when the system grows too large. This could probably be improved by optimizing the code such that the computational basis is always minimal w.r.t. the already evaluated symmetries.